In [12]:
from typing import Type
import open3d as o3d
import numpy as np

from inspect_pc import threshold_pc_distance
from inspect_pc import visualize_two_pc
from inspect_pc import visualize_pc

# Import, visualize and perform spatial filtering

In [27]:
pc_source = o3d.io.read_point_cloud("./measurements/pumpkin/20230717-190933.pcd")
pc_target = o3d.io.read_point_cloud("./measurements/pumpkin/20230717-190946.pcd")

In [28]:
pc_source = threshold_pc_distance(pc_source, 250, 750)
pc_target = threshold_pc_distance(pc_target, 250, 750)

In [29]:
visualize_pc(pc_source, 1.5, "Thresholded point cloud")

In [30]:
visualize_pc(pc_target, 1.5, "Thresholded point cloud")

# Translate & estimate normals

In [38]:
translation_vector = pc_source.get_center() - pc_target.get_center()
pc_target.translate(translation_vector)

PointCloud with 78402 points.

In [39]:
# Estimate pc normals.
radius_normal = 0.005  # 5 cm.
pc_source.estimate_normals(
    o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30)
)
pc_target.estimate_normals(
    o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30)
)

# Register the clouds

In [ ]:
loss = o3d.pipelines.registration.TukeyLoss(k=0.1)
result = o3d.pipelines.registration.registration_icp(
    pc_source,
    pc_target,
    0.01,
    np.identity(4),
    o3d.pipelines.registration.TransformationEstimationPointToPlane(loss),
)

In [ ]:
print(result)

In [42]:
visualize_two_pc(pc_source, pc_target, 1.5, "Initial state")

In [ ]:
visualize_two_pc(pc_source, pc_target, 1.5, "ICP", result.transformation)